##  Create a csv for Training


In [3]:
import pandas as pd
import os
import numpy as np
import glob

#### Read all paths

In [87]:
# batch_2_mask_root = '/fast_data_e2e11/piyush/fake_data/all_fake_masks'
# batch_2_csv_root = '/fast_data_e2e11/piyush/fake_data/fake_data_csvs_batch_2'

batch_3_mask_root = '/fast_data_e2e11/piyush/fake_data/fake_data_batch_3/fake_masks'
batch_3_csv_root = '/fast_data_e2e11/piyush/fake_data/fake_data_batch_3/csvs'
old_datap = '/fast_data_e2e_1/cxr/qxr_ln_data/cxr_data/filt_csvs/train_tag_ex_all_03_08_24.csv'
#normal_imgsp = '/home/users/piyush.kumar/qxr_lungcancer/datasets/LN_csvs/nod_neg_train_val_split/train_negs.csv'

In [88]:

## old csv
old_df = pd.read_csv(old_datap)
old_df = old_df[old_df['big_bbox'] == False]
unique_filenames = old_df['filename'].unique().tolist()
filenames = unique_filenames

## Remove .png from names
filenames = [filename[:-4] for filename in filenames]

# old_data = pd.DataFrame({
#     'filename': filenames,
#     'nodule': [1] * len(filenames),
#     'nipple_shadow': [0] * len(filenames),
#     'normal': [0] * len(filenames),
#     'real' : [1] * len(filenames)

# })

In [89]:
old_training_df = pd.read_csv('/fast_data_e2e11/piyush/training/training_csvs/old_train_data_1.8M.csv')

In [90]:
old_training_df.shape

(1894018, 14)

In [91]:
old_training_df_included =old_training_df[old_training_df.filename.isin(filenames)]
old_training_df_included = pd.concat([old_training_df_included, old_training_df[old_training_df['nodule']==0]])
old_training_df_included.shape

(1852567, 14)

In [92]:
old_training_df.normal.value_counts()

normal
0    1056376
1     837642
Name: count, dtype: int64

In [93]:
old_training_df_included['real'] = 1
old_training_df_included = old_training_df_included[['filename', 'nodule', 'real', 'regular_border', 'irregular_border', 'homogenous', 'inhomogenous', 'normal']]

In [100]:
## collate everything from batch 3
all_batch_3_csvs = glob.glob(batch_3_csv_root + '/*', recursive=True)
df_batch_3 = pd.concat([pd.read_csv(file) for file in all_batch_3_csvs])

### Filter out subtle nodules from the fake images
subtle_nodule_df = df_batch_3[df_batch_3['model_score'] < 0.15]
#subtle_nodule_df[subtle_nodule_df["div"] < 20].shape

In [101]:
irregular_nodule_masks = subtle_nodule_df[subtle_nodule_df['skew'] == True]['filename'].unique().tolist()
regular_nodule_masks = subtle_nodule_df[subtle_nodule_df['skew'] == False]['filename'].unique().tolist()
homogenous_nodule_masks = subtle_nodule_df[subtle_nodule_df['div'] > 30]['filename'].unique().tolist()
inhomo_nodule_masks = subtle_nodule_df[subtle_nodule_df['div'] < 25]['filename'].unique().tolist()

In [18]:
## given a list of filenames, source folder and destination folder, copy the files from source to destination
# def copy_files(filenames, src_folder, dest_folder):
#     for filename in filenames:
#         src = os.path.join(src_folder, filename)
#         dest = os.path.join(dest_folder, filename)
#         os.system('cp ' + src + ' ' + dest)

In [22]:
## copy the files
# copy_files(irregular_nodule_masks, batch_3_mask_root + '/nodule', batch_3_mask_root + '/irregular_border')
# copy_files(regular_nodule_masks, batch_3_mask_root + '/nodule', batch_3_mask_root + '/regular_border')
# copy_files(homogenous_nodule_masks, batch_3_mask_root + '/nodule', batch_3_mask_root + '/homogenous')
# copy_files(inhomo_nodule_masks, batch_3_mask_root + '/nodule', batch_3_mask_root + '/inhomogenous')

In [102]:
### collate everything and create a final training csv
subtle_nodule_df.loc[:, 'normal'] = 0
subtle_nodule_df.loc[:, 'nodule'] = 1
subtle_nodule_df.loc[:, 'real'] = 0
subtle_nodule_df.loc[:, 'homogenous'] = np.where(subtle_nodule_df['filename'].isin(homogenous_nodule_masks), 1, np.where(subtle_nodule_df['filename'].isin(inhomo_nodule_masks), 0, -100))
subtle_nodule_df.loc[:, 'irregular_border'] = np.where(subtle_nodule_df['filename'].isin(irregular_nodule_masks), 1, 0)
subtle_nodule_df.loc[:, 'regular_border'] =  np.where(subtle_nodule_df['filename'].isin(regular_nodule_masks), 1, 0)
subtle_nodule_df.loc[:, 'inhomogenous'] = np.where(subtle_nodule_df['filename'].isin(homogenous_nodule_masks), 1, np.where(subtle_nodule_df['filename'].isin(inhomo_nodule_masks), 0, -100))
subtle_nodule_df.loc[:, 'filename'] = subtle_nodule_df['filename'].apply(lambda x: x[:-4])

/tmp/ipykernel_288989/3928678571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subtle_nodule_df.loc[:, 'normal'] = 0
/tmp/ipykernel_288989/3928678571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subtle_nodule_df.loc[:, 'nodule'] = 1
/tmp/ipykernel_288989/3928678571.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [103]:
## change the column names regular:regular_border, irregular:irregular_border, homogenous:homogenous, inhomogenous:inhomogenous
# subtle_nodule_df = subtle_nodule_df.rename(columns={'regular': 'regular_border', 'irregular': 'irregular_border', 'homogenous': 'homogenous', 'inhomogenous': 'inhomogenous'})
subtle_nodule_df = subtle_nodule_df[['filename', 'nodule', 'real', 'regular_border', 'irregular_border', 'homogenous', 'inhomogenous', 'normal']]
#final_training_df = pd.concat([old_training_df, subtle_nodule_df])
## sanity check
print(subtle_nodule_df.shape)
print(subtle_nodule_df['filename'].nunique())
print(subtle_nodule_df.nodule.value_counts())
print(subtle_nodule_df.real.value_counts())
print(subtle_nodule_df.homogenous.value_counts())
print(subtle_nodule_df.irregular_border.value_counts())
print(subtle_nodule_df.regular_border.value_counts())
print(subtle_nodule_df.inhomogenous.value_counts())
#print(subtle_nodule_df.normal.value_counts())

(43773, 8)
43773
nodule
1    43773
Name: count, dtype: int64
real
0    43773
Name: count, dtype: int64
homogenous
 0      19179
 1      17443
-100     7151
Name: count, dtype: int64
irregular_border
1    24715
0    19058
Name: count, dtype: int64
regular_border
0    24715
1    19058
Name: count, dtype: int64
inhomogenous
 0      19179
 1      17443
-100     7151
Name: count, dtype: int64


In [104]:
subtle_nodule_df.columns

Index(['filename', 'nodule', 'real', 'regular_border', 'irregular_border',
       'homogenous', 'inhomogenous', 'normal'],
      dtype='object')

In [105]:
final_training_df = pd.concat([old_training_df_included, subtle_nodule_df])

In [106]:
## sanity check
print(final_training_df.shape)
print(final_training_df['filename'].nunique())
print(final_training_df.nodule.value_counts())
print(final_training_df.real.value_counts())
print(final_training_df.homogenous.value_counts())
print(final_training_df.irregular_border.value_counts())
print(final_training_df.regular_border.value_counts())
print(final_training_df.inhomogenous.value_counts())
print(final_training_df.normal.value_counts())

(1896340, 8)
1896340
nodule
0    1829287
1      67053
Name: count, dtype: int64
real
1    1852567
0      43773
Name: count, dtype: int64
homogenous
 0      1866941
 1        22083
-100       7316
Name: count, dtype: int64
irregular_border
0    1869454
1      26886
Name: count, dtype: int64
regular_border
0    1861559
1      34781
Name: count, dtype: int64
inhomogenous
 0      1864755
 1        23326
-100       8259
Name: count, dtype: int64
normal
0    1058698
1     837642
Name: count, dtype: int64


In [107]:
final_training_df.columns

Index(['filename', 'nodule', 'real', 'regular_border', 'irregular_border',
       'homogenous', 'inhomogenous', 'normal'],
      dtype='object')

In [108]:
final_training_df.to_csv('/fast_data_e2e11/piyush/training/training_csvs/17_aug_1.8M_full_data.csv', index=False)

In [2]:
full_training_df = pd.read_csv('/fast_data_e2e11/piyush/training/training_csvs/17_aug_1.8M_full_data.csv')

(0, 8)

In [85]:
## print the value counts for each of the columns in the full_training_df
print(full_training_df.nodule.value_counts())
print(full_training_df.real.value_counts())
print(full_training_df.homogenous.value_counts())
print(full_training_df.irregular_border.value_counts())
print(full_training_df.regular_border.value_counts())
print(full_training_df.inhomogenous.value_counts())
print(full_training_df.normal.value_counts())


nodule
0    1829287
1      67053
Name: count, dtype: int64
real
1    1852567
0      43773
Name: count, dtype: int64
homogenous
 0      1866941
 1        22083
-100       7316
Name: count, dtype: int64
irregular_border
0    1869454
1      26886
Name: count, dtype: int64
regular_border
0    1861559
1      34781
Name: count, dtype: int64
inhomogenous
 0      1864755
 1        23326
-100       8259
Name: count, dtype: int64
normal
0.0    1014925
1.0     837642
Name: count, dtype: int64


In [1]:
def get_loss_wts(df, sampling_tags):
    #df = pd.read_csv(args.model.path.path_to_img_df, index_col='path')
    #df["nota"] = df.drop(columns=['split']).sum(axis=1).apply(lambda x: 1 if x == 0 else 0)
    sample_wts = {}
    for each_class in sampling_tags:
        sample_wts[each_class] = 1 / np.sqrt(
            df.loc[df[each_class] != -100, each_class].sum()
            + 0.000001
        )
    min_prob = min(sample_wts.values())
    loss_wts = {k: x / min_prob for k, x in sample_wts.items()}
    return loss_wts

In [4]:
final_training_df = pd.read_csv("/raid/piyush/training_csvs/31_aug_2M_with_3d_data_no_real_nodules.csv")

In [8]:
cls_sampling_tags = ['nodule', 'regular_border', 'irregular_border', 'homogenous', 'inhomogenous','calcified' ,'artifact',  'normal']
cls_loss_wts = get_loss_wts(final_training_df, cls_sampling_tags)

seg_sampling_tags = ['nodule', 'regular_border', 'irregular_border', 'homogenous', 'inhomogenous', 'calcified' ]
seg_loss_wts = get_loss_wts(final_training_df, seg_sampling_tags)


In [9]:
cls_loss_wts, seg_loss_wts

({'nodule': 2.891839576025378,
  'regular_border': 3.3878593562272283,
  'irregular_border': 6.1795705272134045,
  'homogenous': 4.23818156079613,
  'inhomogenous': 4.956110225848023,
  'calcified': 6.4613864125693246,
  'artifact': 6.405788573422099,
  'normal': 1.0},
 {'nodule': 1.0,
  'regular_border': 1.171523961534406,
  'irregular_border': 2.1368994941644623,
  'homogenous': 1.4655659310884737,
  'inhomogenous': 1.7138261288545729,
  'calcified': 2.234351610005292})

### Result Analysis

In [5]:
nlst_jsrt_df = pd.read_csv('/fast_data_e2e11/qxr_ln_trainings/testing/nlst_jsrt.csv')

nlst_jsrt_df.columns

Index(['filename', 'nodule', 'nipple_shadow', 'normal', 'homogenous',
       'inhomogenous', 'irregular_border', 'regular_border'],
      dtype='object')

In [18]:
swin_base_17_aug_rs_df = pd.read_csv('/fast_data_e2e11/qxr_ln_trainings/checkpoints/results/base_training/17_aug_swin_base_full_data/model_1044-last/model_1044-last.csv')
swin_tiny_17_aug_rs_df = pd.read_csv('/fast_data_e2e11/qxr_ln_trainings/checkpoints/results/base_training/17_aug_swin_tiny_full_data/model_993-last/model_993-last.csv')

In [11]:
## Given csv with column nodule and nodule_target, calculate auc, yoden index, sensitivity, specificity, false positives, false negatives
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


#false positive rate, false negative rate, sensitivity, specificity, precision, recall, f1 score, accuracy, auc, 
#return it in the form of a dictionary


def get_metrics(df, target_col, pred_col):
    fpr, tpr, thresholds = roc_curve(df[target_col], df[pred_col])
    auc = roc_auc_score(df[target_col], df[pred_col])
    yoden_index = tpr - fpr
    best_threshold = thresholds[np.argmax(yoden_index)]
    yoden_index = np.max(yoden_index)
    sensitivity = recall_score(df[target_col], df[pred_col] > best_threshold)
    specificity = 1 - fpr[np.argmax(tpr > sensitivity)]
    precision = precision_score(df[target_col], df[pred_col] > best_threshold)
    recall = sensitivity
    f1 = f1_score(df[target_col], df[pred_col] > best_threshold)
    accuracy = accuracy_score(df[target_col], df[pred_col] > best_threshold)
    false_positives = confusion_matrix(df[target_col], df[pred_col] > best_threshold)[0][1]
    false_negatives = confusion_matrix(df[target_col], df[pred_col] > best_threshold)[1][0]
    return {
        'fpr': fpr,
        'tpr': tpr,
        'thresholds': thresholds,
        'auc': auc,
        'yoden_index': yoden_index,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'false_positives': false_positives,
        'false_negatives': false_negatives
    }


## get the metrics for the nlst_jsrt_df


In [19]:
swin_tiny_metrics = get_metrics(swin_tiny_17_aug_rs_df, 'nodule_target', 'nodule')
swin_base_metrics = get_metrics(swin_base_17_aug_rs_df, 'nodule_target', 'nodule')

In [21]:
swin_tiny_metrics

{'fpr': array([0.        , 0.00166113, 0.00332226, 0.00332226, 0.01328904,
        0.01328904, 0.0166113 , 0.0166113 , 0.01993355, 0.01993355,
        0.02325581, 0.02325581, 0.02491694, 0.02491694, 0.02657807,
        0.02657807, 0.02990033, 0.02990033, 0.03322259, 0.03322259,
        0.04318937, 0.04318937, 0.0448505 , 0.0448505 , 0.04651163,
        0.04651163, 0.05149502, 0.05149502, 0.05315615, 0.05315615,
        0.05481728, 0.05481728, 0.05647841, 0.05647841, 0.05813953,
        0.05813953, 0.05980066, 0.05980066, 0.06478405, 0.06478405,
        0.06810631, 0.06810631, 0.07142857, 0.07142857, 0.07475083,
        0.07475083, 0.07641196, 0.07641196, 0.07973422, 0.07973422,
        0.08471761, 0.08471761, 0.08637874, 0.08637874, 0.09136213,
        0.09136213, 0.10299003, 0.10299003, 0.10465116, 0.10465116,
        0.10631229, 0.10631229, 0.10963455, 0.10963455, 0.11129568,
        0.11129568, 0.11461794, 0.11461794, 0.11960133, 0.11960133,
        0.12624585, 0.12624585, 0.1345515

> Calculate sampling weights

In [ ]:
# def get_loss_wts(df):
    
#     #df["nota"] = df.drop(columns=['split']).sum(axis=1).apply(lambda x: 1 if x == 0 else 0)
#     sample_wts = {}
#     all_tags = df.columns
#     for each_class in all_tags:
#         sample_wts[each_class] = 1 / np.sqrt(
#             df.loc[df[each_class] != -100, each_class].sum()
#             + 0.000001
#         )
#     min_prob = min(sample_wts.values())
#     loss_wts = {k: x / min_prob for k, x in sample_wts.items()}
#     return loss_wts


# loss_wts = get_loss_wts(training_df)
# loss_wts
# def get_loss_wts(df):
#     # Vectorized computation of sample_wts
#     sample_wts = 1 / np.sqrt((df[df != -100].sum() + 0.000001))
#     sample_wts = sample_wts.to_dict()

#     min_prob = min(sample_wts.values())
#     loss_wts = {k: x / min_prob for k, x in sample_wts.items()}
#     return loss_wts

# loss_wts = get_loss_wts(training_df)
# loss_wts

In [ ]:
training_df.nodule.value_counts()

nodule
0    700404
1    134116
Name: count, dtype: int64

In [ ]:
sample_wts = {'nodule': 1/np.sqrt(134916), 'nipple_shadow': 1/np.sqrt(30005)}#, 'normal': 1/np.sqrt(670399)}
min_prob = min(sample_wts.values())
loss_wts = {k: x / min_prob for k, x in sample_wts.items()}
loss_wts


{'nodule': 1.0, 'nipple_shadow': 2.1204835749347533}

In [ ]:
auc_swin_tiny = '/fast_data_e2e11/qxr_ln_trainings/checkpoints/results/base_training/12_Aug_swin_tiny_base_unetplusplus_sq_dice_loss/auc_.csv'

auc_swin_tiny_df = pd.read_csv(auc_swin_tiny)
auc_swin_tiny_df

,model_name,AUROC,nodule_AUROC,nodule_youden_index_cuttoff,nipple_shadow,normal,AUPRC,nodule_AUPRC,F1_score,Fbeta_score,...,accuracy,nodule_sensitivity,nodule_specificity,nodule_precision,nodule_accuracy,nodule_NPV,nodule_tp,nodule_tn,nodule_fp,nodule_fn
0,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.760259,0.760259,0.149837,0.1,0.1,0.566330,0.566330,0.0,0.0,...,0.740560,0.657534,0.770764,0.510638,0.740560,0.860853,144.0,464.0,138.0,75.0
1,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.744580,0.744580,0.226599,0.1,0.1,0.513410,0.513410,0.0,0.0,...,0.742996,0.593607,0.797342,0.515873,0.742996,0.843585,130.0,480.0,122.0,89.0
2,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.744580,0.744580,0.226599,0.1,0.1,0.513410,0.513410,0.0,0.0,...,0.742996,0.593607,0.797342,0.515873,0.742996,0.843585,130.0,480.0,122.0,89.0
3,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.722333,0.722333,0.041365,0.1,0.1,0.495954,0.495954,0.0,0.0,...,0.702801,0.634703,0.727575,0.458746,0.702801,0.845560,139.0,438.0,164.0,80.0


In [ ]:
auc_swin_base = '/fast_data_e2e11/qxr_ln_trainings/checkpoints/results/base_training/12_Aug_swin_base_unetplusplus_dice_loss/auc_.csv'
auc_swin_base_df = pd.read_csv(auc_swin_base)
auc_swin_base_df

,model_name,AUROC,nodule_AUROC,nodule_youden_index_cuttoff,nipple_shadow,normal,AUPRC,nodule_AUPRC,F1_score,Fbeta_score,...,accuracy,nodule_sensitivity,nodule_specificity,nodule_precision,nodule_accuracy,nodule_NPV,nodule_tp,nodule_tn,nodule_fp,nodule_fn
0,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.761601,0.761601,0.142791,0.1,0.1,0.548148,0.548148,0.0,0.0,...,0.753959,0.662100,0.787375,0.531136,0.753959,0.864964,145.0,474.0,128.0,74.0
1,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.745309,0.745309,0.146365,0.1,0.1,0.527668,0.527668,0.0,0.0,...,0.747868,0.589041,0.805648,0.524390,0.747868,0.843478,129.0,485.0,117.0,90.0
2,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.738437,0.738437,0.080545,0.1,0.1,0.535162,0.535162,0.0,0.0,...,0.764921,0.593607,0.827243,0.555556,0.764921,0.848382,130.0,498.0,104.0,89.0
3,/fast_data_e2e11/qxr_ln_trainings/checkpoints/...,0.726649,0.726649,0.221309,0.1,0.1,0.485359,0.485359,0.0,0.0,...,0.750305,0.552511,0.822259,0.530702,0.750305,0.834739,121.0,495.0,107.0,98.0


In [ ]:
auc_swin_base_df.model_name[1]

'/fast_data_e2e11/qxr_ln_trainings/checkpoints/base_training/12_Aug_swin_base_unetplusplus_dice_loss/model_1147-epoch=54-val_loss=1.628382.ckpt'

In [ ]:
test_csv = '/fast_data_e2e11/qxr_ln_trainings/testing/nlst_jsrt.csv'
test_df = pd.read_csv(test_csv)
test_df

,filename,nodule,nipple_shadow,normal
0,200226,1,-100,-100
1,200314,1,-100,-100
2,200323,1,-100,-100
3,200442,0,-100,-100
4,200565,1,-100,-100
...,...,...,...,...
816,JPCNN089,0,-100,-100
817,JPCNN090,0,-100,-100
818,JPCNN091,0,-100,-100
819,JPCNN092,0,-100,-100


In [ ]:
import timm

/home/users/piyush.kumar/miniconda3/envs/qtrain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


#efficientnetv2_m

bat_resnext26ts
beit_base_patch16_224
beit_base_patch16_384
beit_large_patch16_224
beit_large_patch16_384
beit_large_patch16_512
beitv2_base_patch16_224
beitv2_large_patch16_224
botnet26t_256
botnet50ts_256
caformer_b36
caformer_m36
caformer_s18
caformer_s36
cait_m36_384
cait_m48_448
cait_s24_224
cait_s24_384
cait_s36_384
cait_xs24_384
cait_xxs24_224
cait_xxs24_384
cait_xxs36_224
cait_xxs36_384
coat_lite_medium
coat_lite_medium_384
coat_lite_mini
coat_lite_small
coat_lite_tiny
coat_mini
coat_small
coat_tiny
coatnet_0_224
coatnet_0_rw_224
coatnet_1_224
coatnet_1_rw_224
coatnet_2_224
coatnet_2_rw_224
coatnet_3_224
coatnet_3_rw_224
coatnet_4_224
coatnet_5_224
coatnet_bn_0_rw_224
coatnet_nano_cc_224
coatnet_nano_rw_224
coatnet_pico_rw_224
coatnet_rmlp_0_rw_224
coatnet_rmlp_1_rw2_224
coatnet_rmlp_1_rw_224
coatnet_rmlp_2_rw_224
coatnet_rmlp_2_rw_384
coatnet_rmlp_3_rw_224
coatnet_rmlp_nano_rw_224
coatnext_nano_rw_224
convformer_b36
convformer_m36
convformer_s18
convformer_s36
convit_base
conv

In [1]:
import pandas as pd

In [2]:
sahil_df = pd.read_csv('/fast_data_2d_3/cxr_data/training/csvs/training_master_filt_07-09-23.csv')

In [5]:
for col in sahil_df.columns:
    print(col)

filename
abcess
airbronchogram
aircrescent
airtrapping
apicalcap
aorticcalcification
atelectasis
azygosfissure
bluntedcp
border
bronchiectasis
bronchitis
bronchocele
bulla
bullousemphysema
calcification
carcinoma
cardiomegaly
cavity
consolidation
cyst
degenspine
diaphragmborder
edema
effusion
emphysema
fibrosis
justfibrosis
fissuralshift
fissuralthickening
fracture
oldfracture
ribfracture
ribfractureall
claviclefracture
femurfracture
granuloma
groundglass
haziness
heartborder
hydropneumothorax
interlobularseptalthickening
interstitialthickening
juxtaphrenicpeak
kerley
lesion
bullouslesion
lymphadenopathy
markings
mass
bonemetastasis
metastases
mediastinialshift
miliarypattern
nodule
nonaorticcalcification
oligemia
opacity
density
thickening
osteopenia
pacemaker
peffusion
peribronchialthickening
pleuralplaque
pleuralthickening
pneumatocele
pneumomediastinum
pneumonia
pneumopericardium
pneumothorax
pulmonaryfibrosis
raiseddiaphragm
tenteddiaphragm
humpdiaphragm
scoliosis
shift
suture
ten